In [1]:
import pandas as pd
import sqlalchemy
import geopandas as gpd
import seaborn as sns
from shapely import wkt
from itertools import cycle, islice

In [2]:
df = pd.read_csv(r'..\..\..\outputs\network\network_results.csv')

# Congested network components by time of day
df.columns

# Get freeflow from 20to5 period

# Exclude trips taken on non-designated facilities (facility_type == 0)
# These are artificial (weave lanes to connect HOV) or for non-auto uses 
df = df[df['data3'] != 0]    # data3 represents facility_type

# calculate total link VMT and VHT
df['VMT'] = df['@tveh']*df['length']
df['VHT'] = df['@tveh']*df['auto_time']/60

# Define facility type
df.loc[df['data3'].isin([1,2]), 'facility_type'] = 'highway'
df.loc[df['data3'].isin([3,4,6]), 'facility_type'] = 'arterial'
df.loc[df['data3'].isin([5]), 'facility_type'] = 'connector'

# Calculate delay
# Select links from overnight time of day
delay_df = df.loc[df['tod'] == '20to5'][['ij','auto_time']]
delay_df.rename(columns={'auto_time':'freeflow_time'}, inplace=True)

# Merge delay field back onto network link df
df = pd.merge(df, delay_df, on='ij', how='left')

# Calcualte hourly delay
df['total_delay'] = ((df['auto_time']-df['freeflow_time'])*df['@tveh'])/60    # sum of (volume)*(travtime diff from freeflow)

# Calcualte speed as a percent of freeflow
df['flow_index'] = df['auto_time']/df['freeflow_time']

df['county'] =df['@countyid'].map({33: 'King',
                                      35: 'Kitsap',
                                      53: 'Pierce',
                                      61: 'Snohomish'})

df['speed'] = df['length']/df['auto_time']*60
df['congestion_index'] = df['speed']/df['data2']
df['congestion_index'] = df['congestion_index'].clip(0,1)
df['congestion_category'] = pd.cut(df['congestion_index'], bins=[0,.25,.5,.7,1], labels=['Severe','Heavy','Moderate','Light'])

In [3]:
def read_from_sde(connection_string, feature_class_name, version,
                  crs={'init': 'epsg:2285'}, is_table = False):
    """
    Returns the specified feature class as a geodataframe from ElmerGeo.
    
    Parameters
    ----------
    connection_string : SQL connection string that is read by geopandas 
                        read_sql function
    
    feature_class_name: the name of the featureclass in PSRC's ElmerGeo 
                        Geodatabase
    
    cs: cordinate system
    """


    engine = sqlalchemy.create_engine(connection_string)
    con=engine.connect()
    #con.execute("sde.set_current_version {0}".format(version))
    if is_table:
        gdf=pd.read_sql('select * from %s' % 
                   (feature_class_name), con=con)
        con.close()

    else:
        df=pd.read_sql('select *, Shape.STAsText() as geometry from %s' % 
                   (feature_class_name), con=con)
        con.close()

        df['geometry'] = df['geometry'].apply(wkt.loads)
        gdf=gpd.GeoDataFrame(df, geometry='geometry')
        gdf.crs = crs
        cols = [col for col in gdf.columns if col not in 
                ['Shape', 'GDB_GEOMATTR_DATA', 'SDE_STATE_ID']]
        gdf = gdf[cols]
    
    return gdf


In [4]:
def format_df(df, col_list, col_name, index_list, index_name):
    
    # Re-order columns
    df = df[col_list]
    # Re-order index
    df = df.reindex(index_list)
    # Set visible column and index names
    df.columns.name = col_name
    df.index.name = index_name
    
    return df


In [5]:
# set chart style
sns.set_style("dark")
sns.set_context("notebook", font_scale=1.5, 
                rc={"lines.linewidth": 1.5})
colors = ['#91268F','#F05A28','#8CC63E','#00A7A0','#4C4C4C', '#BFE9E7']
sns.set(font="Pragmatica Condensed Light")

In [6]:
# Load regional geographies layer to intersect with network shapefile
connection_string = 'mssql+pyodbc://AWS-PROD-SQL\Sockeye/ElmerGeo?driver=SQL Server?Trusted_Connection=yes'
crs = {'init' : 'EPSG:2285'}
version = "'DBO.Default'"
gdf_shp = read_from_sde(connection_string, 'urban_centers', version, crs=crs, is_table=False)

# Intersect geography this with the network shapefile
gdf_network = gpd.read_file(r'..\..\..\inputs\scenario\networks\shapefiles\AM\AM_edges.shp')
gdf_intersect = gpd.overlay(gdf_network, gdf_shp, how="union", keep_geom_type=False)

# Will need to relaculate the lengths since some were split across the regional geographies
gdf_intersect['new_length'] = gdf_intersect.geometry.length/5280.0

### IMPORTANT
# filter out the polygon results and only keep lines
gdf_intersect = gdf_intersect[gdf_intersect.geometry.type.isin(['MultiLineString','LineString'])]

C:\Users\modeller\Anaconda3\envs\summary\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [7]:
df = df[['@tveh','congestion_category','i_node','j_node','tod','total_delay']].merge(gdf_intersect, 
                                                             left_on=['i_node','j_node'],
                                                            right_on=['i','j'],
                                                                  how='left')
df['VMT'] = df['new_length']*df['@tveh']

# Assume speed is constant or assigned link
# When links are split, we take proportionate delay by length

# calculate total delay
df['delay'] = (df['new_length']/df['length'])*df['total_delay']
# Note that we won't get all of the data because some links are outside the region boundaries
# Optionally, scale these numbers by the total delay field total_delay for consistency
df['scaled_delay'] = df['delay']*(df['total_delay'].sum()/df['delay'].sum())

# Congestion by Regional Geography
### Total VMT & Percent of VMT by Congestion Level

- **Severe: < 25% Speed Limit** (e.g., less than 15 mph on a road with speed limit of 60 mph)
- **Heavy: 25-50% Speed Limit** (e.g., 15-30 mph on a road with speed limit of 60 mph)
- **Moderate: 50-70% Speed Limit** (e.g, 30-42 mph on a road with speed limit of 60 mph)

#### All Day

In [8]:
df.rename(columns={'name': 'Regional Growth Center'}, inplace=True)
df['Regional Growth Center'] = df['Regional Growth Center'].fillna('Not in RGC')

# Percent of VMT by congestion category
pd.options.display.float_format = '{:0,.0f}'.format
_df = df.pivot_table(index='congestion_category',columns='Regional Growth Center',
               aggfunc='sum',values='VMT')
_df = _df.T
_df.loc['Total',:] = _df.sum() 
_df

congestion_category,Severe,Heavy,Moderate,Light
Regional Growth Center,,,,
Auburn,285,"4,017","9,406","38,200"
Bellevue,"1,208","8,710","18,438","67,009"
Bothell Canyon Park,"4,554","17,001","14,282","146,625"
Bremerton,69,815,"2,557","23,620"
Burien,238,"2,361","8,759","64,237"
Everett,143,"7,341","16,423","62,498"
Federal Way,76,"1,052","3,594","21,959"
Greater Downtown Kirkland,653,"7,180","14,721","113,612"
Issaquah,"1,850","3,900","9,260","178,080"


In [9]:
pd.options.display.float_format = '{:0,.1%}'.format
_df = _df.drop('Total', axis=0)
_sum = _df.sum(axis=1)
for col in _df.columns.categories:
    _df[col] = _df[col]/_sum
_df

congestion_category,Severe,Heavy,Moderate,Light
Regional Growth Center,,,,
Auburn,0.5%,7.7%,18.1%,73.6%
Bellevue,1.3%,9.1%,19.3%,70.3%
Bothell Canyon Park,2.5%,9.3%,7.8%,80.4%
Bremerton,0.3%,3.0%,9.4%,87.3%
Burien,0.3%,3.1%,11.6%,85.0%
Everett,0.2%,8.5%,19.0%,72.3%
Federal Way,0.3%,3.9%,13.5%,82.3%
Greater Downtown Kirkland,0.5%,5.3%,10.8%,83.4%
Issaquah,1.0%,2.0%,4.8%,92.2%


#### AM Peak (7 to 8 AM)

In [10]:
pd.options.display.float_format = '{:0,.0f}'.format
df_am = df[df['tod'] == '7to8']
_df = df_am.pivot_table(index='congestion_category',columns='Regional Growth Center',
               aggfunc='sum',values='VMT')
_df = _df.T
_df.loc['Total',:] = _df.sum() 
_df

congestion_category,Severe,Heavy,Moderate,Light
Regional Growth Center,,,,
Auburn,25,394,"1,337","1,809"
Bellevue,66,533,"1,095","4,544"
Bothell Canyon Park,228,"2,574","1,909","6,708"
Bremerton,2,94,167,"1,893"
Burien,17,176,433,"3,951"
Everett,18,555,993,"3,941"
Federal Way,5,6,289,"1,189"
Greater Downtown Kirkland,45,"1,878","1,286","6,561"
Issaquah,253,318,"2,305","11,907"


In [11]:
pd.options.display.float_format = '{:0,.1%}'.format
_df = _df.drop('Total', axis=0)
_sum = _df.sum(axis=1)
for col in _df.columns.categories:
    _df[col] = _df[col]/_sum
_df

congestion_category,Severe,Heavy,Moderate,Light
Regional Growth Center,,,,
Auburn,0.7%,11.1%,37.5%,50.7%
Bellevue,1.1%,8.5%,17.6%,72.8%
Bothell Canyon Park,2.0%,22.5%,16.7%,58.7%
Bremerton,0.1%,4.4%,7.7%,87.8%
Burien,0.4%,3.8%,9.5%,86.3%
Everett,0.3%,10.1%,18.0%,71.6%
Federal Way,0.3%,0.4%,19.4%,79.9%
Greater Downtown Kirkland,0.5%,19.2%,13.2%,67.2%
Issaquah,1.7%,2.1%,15.6%,80.5%


#### PM Peak (5 to 6 PM)

In [12]:
pd.options.display.float_format = '{:0,.0f}'.format
df_pm = df[df['tod'] == '17to18']
_df = df_pm.pivot_table(index='congestion_category',columns='Regional Growth Center',
               aggfunc='sum',values='VMT')
_df = _df.T
_df.loc['Total',:] = _df.sum() 
_df

congestion_category,Severe,Heavy,Moderate,Light
Regional Growth Center,,,,
Auburn,23,460,732,"3,128"
Bellevue,106,"1,160","1,308","5,282"
Bothell Canyon Park,620,889,"2,223","10,147"
Bremerton,40,51,233,"2,049"
Burien,17,268,820,"5,586"
Everett,10,672,"1,285","5,171"
Federal Way,6,115,272,"1,601"
Greater Downtown Kirkland,64,820,"1,699","8,572"
Issaquah,190,380,636,"15,454"


In [13]:
pd.options.display.float_format = '{:0,.1%}'.format
_df = _df.drop('Total', axis=0)
_sum = _df.sum(axis=1)
for col in _df.columns.categories:
    _df[col] = _df[col]/_sum
_df

congestion_category,Severe,Heavy,Moderate,Light
Regional Growth Center,,,,
Auburn,0.5%,10.6%,16.9%,72.0%
Bellevue,1.4%,14.8%,16.6%,67.2%
Bothell Canyon Park,4.5%,6.4%,16.0%,73.1%
Bremerton,1.7%,2.2%,9.8%,86.4%
Burien,0.2%,4.0%,12.3%,83.5%
Everett,0.1%,9.4%,18.0%,72.4%
Federal Way,0.3%,5.8%,13.6%,80.3%
Greater Downtown Kirkland,0.6%,7.3%,15.2%,76.9%
Issaquah,1.1%,2.3%,3.8%,92.8%


# Network Delay
Totals scaled to match network totals. 

In [14]:
# Percent of VMT by congestion category
pd.options.display.float_format = '{:0,.0f}'.format
_df = df.groupby('Regional Growth Center').sum()[['scaled_delay']]
_df.loc['Total',:] = _df.sum() 
_df

,scaled_delay
Regional Growth Center,
Auburn,146
Bellevue,298
Bothell Canyon Park,"1,340"
Bremerton,26
Burien,84
Everett,131
Federal Way,24
Greater Downtown Kirkland,406
Issaquah,542


Totals unscaled. Some links cross outside the region and delay on these links are excluded at boundaries

In [15]:
# Percent of VMT by congestion category
pd.options.display.float_format = '{:0,.0f}'.format
_df = df.groupby('Regional Growth Center').sum()[['delay']]
_df.loc['Total',:] = _df.sum() 
_df

,delay
Regional Growth Center,
Auburn,139
Bellevue,283
Bothell Canyon Park,"1,273"
Bremerton,25
Burien,80
Everett,125
Federal Way,23
Greater Downtown Kirkland,385
Issaquah,515
